<a href="https://colab.research.google.com/github/MariaGorelik/NER_furniture_stores/blob/main/NER_furniture_stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Conversion Script:

In [ ]:
!pip install datasets
!pip install transformers

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00


In [ ]:
import json

# Загрузка данных из JSON файла
with open('corrected_annotations.json', 'r') as file:
    data = json.load(file)

all_text = []
for line in data['annotations']:
  words = line[0].split()
  if not line[1]['entities']:
    for word in words:
      all_text.append(word + ' O')
  else:
    all_text.append(words[0] + ' B-Product')
    if len(words) > 1:
      for i in range(1, len(words)):
        all_text.append(words[i] + ' I-Product')


with open('bio_annotation.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(all_text))

In [ ]:
begin_indices = []

with open('bio_annotation.txt', 'r', encoding='utf-8') as infile:
    for i, line in enumerate(infile):
        if line.split()[-1] == 'B-Product':
          begin_indices.append(i)

for ind in begin_indices:
  print(ind)

0
3830
3906
3936
3967
3995
4025
4055
4081
4145
4301
4388
4701
4726
4742
4758
4771
4784
4797
4813
4823
5629
5645
5661
5677
5693
5709
5725
5742
5765
5785
5805
5829
5847
6309
6317
6331
6338
6347
6355
6365
6370
6379
6387
6395
6405
6412
6419
6427
6434
6442
6450
6458
6467
6475
6477
6485
6489
6495
6499
7370
7378
7453
7459
7631
7660
7666
7671
7679
7686
7694
7825
7831
7837
7843
7850
8016
8158
8172
8532
8679
10832
10834
12029
12032
12050
12438
12456
12470
12484
12497
12514
12528
12543
12557
12571
12585
12599
12613
12627
12640
12650
12664
12679
12695
12709
12723
12737
12747
12759
12772
12784
12796
12809
12822
12835
12849
12863
12876
12889
12902
12916
12929
12943
12957
12970
12983
12997
13011
13024
13038
13051
13064
13286
13291
13296
13301
13306
13311
13316
13320
13324
13329
13335
13341
13347
13353
13359
13365
13483
13504
13510
13515
13520
13525
13529
13534
13539
13544
13549
13554
13558
13563
13567
13572
13576
13579
13583
13588
13592
13697
13756
13776
13790
13805
13818
13826
13841
13851
13859
1562

In [ ]:
import json

# Загрузка данных из JSON файла
with open('corrected_annotations.json', 'r') as file:
    data = json.load(file)

# Преобразование аннотаций в формат (text, label)
train_data = []

for annotation in data['annotations']:
    text = annotation[0]  # Текст
    entities = annotation[1]['entities']

    # Если есть аннотации для продукта, метка = 1, иначе 0
    label = 1 if entities else 0
    train_data.append((text, label))

# Пример сохранения в CSV
import pandas as pd

df = pd.DataFrame(train_data, columns=['text', 'label'])
df.to_csv('train_data.csv', index=False)


In [ ]:
!pip install transformers pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Загрузка данных
df = pd.read_csv('train_data.csv')

# Разделение данных на обучающую и тестовую выборки
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Токенизация текстов
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

# Преобразование в тензоры
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ProductDataset(train_encodings, train_labels.tolist())
val_dataset = ProductDataset(val_encodings, val_labels.tolist())

# Проверяем, есть ли доступ к CUDA (GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

# Инициализация модели и перенос на устройство (GPU или CPU)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)  # Явно передаем модель на устройство

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',           # Куда сохранять результаты
    num_train_epochs=3,               # Количество эпох
    per_device_train_batch_size=8,    # Размер батча на одно устройство (GPU или CPU)
    per_device_eval_batch_size=8,     # Размер батча для оценки
    warmup_steps=500,                 # Шаги для прогрева
    weight_decay=0.01,                # Коэффициент регуляризации
    logging_dir='./logs',             # Директория для логов
    evaluation_strategy="epoch"      # Оценка после каждой эпохи
)

# Создание Trainer
trainer = Trainer(
    model=model,                         # Модель для тренировки
    args=training_args,                  # Параметры обучения
    train_dataset=train_dataset,         # Обучающий набор данных
    eval_dataset=val_dataset             # Валидационный набор данных
)

# Обучение модели
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.016100,0.000555
2,0.004300,0.000022


Epoch,Training Loss,Validation Loss
1,0.016100,0.000555
2,0.004300,0.000022
3,0.000000,0.000033


TrainOutput(global_step=5925, training_loss=0.014256246856684926, metrics={'train_runtime': 4426.7988, 'train_samples_per_second': 10.705, 'train_steps_per_second': 1.338, 'total_flos': 1.246909602456576e+16, 'train_loss': 0.014256246856684926, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Создание директории на Google Диске для сохранения модели
output_dir = '/content/drive/MyDrive/bert_product_model/'

# Сохранение модели и токенизатора
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/bert_product_model/tokenizer_config.json',
 '/content/drive/MyDrive/bert_product_model/special_tokens_map.json',
 '/content/drive/MyDrive/bert_product_model/vocab.txt',
 '/content/drive/MyDrive/bert_product_model/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Путь к директории с сохранённой моделью на Google Диске
model_dir = '/content/drive/MyDrive/bert_product_model/'

# Восстановление модели и токенизатора
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Если доступен GPU, перенесем модель на устройство
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print(device)

cuda


In [ ]:
import requests
from bs4 import BeautifulSoup
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

# Функция для сбора текста с сайта
def scrape_webpage(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Получаем весь текст с сайта
            full_text = soup.get_text(separator='\n')
            # Разделяем текст на строки и убираем пустые
            lines = [line.strip() for line in full_text.split('\n') if line.strip()]
            return lines  # Возвращаем список непустых строк
        else:
            print(f"Failed to retrieve page: {url}, status code={response.status_code}")
            return []
    except Exception as e:
        print(f"Error fetching {url}: {str(e)}")
        return []

# Функция для предсказания
def predict_products(lines, model, tokenizer):
    products = []  # Список для хранения найденных продуктов
    for line in lines:
        # Токенизация строки
        inputs = tokenizer(line, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)

        # Получаем логиты от модели
        with torch.no_grad():
            outputs = model(**inputs)

        # Получаем логиты и предсказываем классы
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()  # Извлекаем предсказание для строки

        # Если предсказание равно 1, добавляем строку в список продуктов
        if prediction == 0:  # Если это продукт
            products.append(line)
            print(line)
            print(logits)

    return products

# Пример использования
url = 'https://dunlin.com.au/products/beadlight-cirrus?variant=724803185'  # Замените на нужный URL
lines = scrape_webpage(url)

# Если есть текст, передаем его в модель для предсказания продуктов
if lines:
    products = predict_products(lines, model, tokenizer)
    # print("Найденные продукты:")
    # for product in products:
    #     print(product)


– Dunlin Home
tensor([[ 4.4446, -4.7786]], device='cuda:0')
Skip to content
tensor([[ 5.0500, -5.4471]], device='cuda:0')
Cart
tensor([[ 5.2991, -5.7248]], device='cuda:0')
$0.00
tensor([[ 5.2812, -5.6939]], device='cuda:0')
(0)
tensor([[ 5.2880, -5.7107]], device='cuda:0')
Search
tensor([[ 5.3013, -5.7247]], device='cuda:0')
Cart
tensor([[ 5.2991, -5.7248]], device='cuda:0')
$0.00
tensor([[ 5.2812, -5.6939]], device='cuda:0')
(0)
tensor([[ 5.2880, -5.7107]], device='cuda:0')
NEW
tensor([[ 5.2962, -5.7227]], device='cuda:0')
AVAILABILITY
tensor([[ 5.3009, -5.7258]], device='cuda:0')
Available Now
tensor([[ 5.3088, -5.7129]], device='cuda:0')
Pre-order
tensor([[ 5.3002, -5.7276]], device='cuda:0')
Made to Order
tensor([[ 5.0961, -5.4855]], device='cuda:0')
OUTDOOR LIGHTS
tensor([[ 5.1233, -5.5217]], device='cuda:0')
STEP LIGHTS
tensor([[ 4.8188, -5.1645]], device='cuda:0')
DAVEY LIGHTING
tensor([[ 5.2959, -5.7028]], device='cuda:0')
INDOOR LIGHTS
tensor([[ 5.1361, -5.5334]], device='cud

Prediction: Product


### По новой!

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
pip install transformers datasets

In [ ]:
data = {
    'tokens': [[]],
    'labels': [[]]  # 0 = O, 1 = B-Product, 2 = I-Product
}

with open('bio_annotation_fixed.txt', 'r', encoding='utf-8') as bio_file:
      for line in bio_file:
        words = line.split()
        data['tokens'][0].append(words[0])
        if words[1] == 'O':
          data['labels'][0].append(0)
        elif words[1] == 'B-Product':
          data['labels'][0].append(1)
        elif words[1] == 'I-Product':
          data['labels'][0].append(2)

In [ ]:
print(len(data['tokens'][0]) == len(data['labels'][0]))

True


In [ ]:
print(data['tokens'][0][:20])
print(data['labels'][0][:20])

['Factory', 'Buys', '32cm', 'Euro', 'Top', 'Mattress', '-', 'King', 'Buys', '32cm', 'Euro', 'Top', 'Mattress', '-', 'King', 'Skip', 'to', 'content', 'FREE', 'SHIPPING']
[1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset

# Используем модель BERT и быстрый токенизатор
model_name = "bert-base-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=3)  # O, B-Product, I-Product

# Превращаем в Dataset
dataset = Dataset.from_dict(data)

# Токенизация данных
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Определяем параметры обучения
training_args = TrainingArguments(
    output_dir="./results",            # Директория для сохранения моделей
    evaluation_strategy="epoch",       # Оценка на каждой эпохе
    per_device_train_batch_size=16,    # Размер батча
    per_device_eval_batch_size=16,     # Размер батча для оценки
    num_train_epochs=12,                # Количество эпох
    weight_decay=0.01,                 # Уменьшение весов для регуляризации
    logging_dir='./logs',              # Директория для логов
)

# Инициализируем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,    # Ваш тренировочный набор
    eval_dataset=tokenized_dataset,     # Ваш тестовый набор (при необходимости)
)

# Запускаем дообучение
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.237431
2,No log,0.091020
3,No log,0.066529
4,No log,0.053018
5,No log,0.045225
6,No log,0.040656
7,No log,0.037362
8,No log,0.034738
9,No log,0.032634
10,No log,0.031013


TrainOutput(global_step=12, training_loss=0.16389252742131552, metrics={'train_runtime': 9.2014, 'train_samples_per_second': 1.304, 'train_steps_per_second': 1.304, 'total_flos': 3135589429248.0, 'train_loss': 0.16389252742131552, 'epoch': 12.0})

In [ ]:
# Сохраняем модель и токенизатор
model.save_pretrained("./saved_model_12")
tokenizer.save_pretrained("./saved_model_12")

('./saved_model_12/tokenizer_config.json',
 './saved_model_12/special_tokens_map.json',
 './saved_model_12/vocab.txt',
 './saved_model_12/added_tokens.json',
 './saved_model_12/tokenizer.json')

In [ ]:
# Заново создаем ваш тренировочный датасет (как и ранее)
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Определяем новые параметры для дообучения
new_training_args = TrainingArguments(
    output_dir="./results",             # Директория для сохранения моделей
    evaluation_strategy="epoch",        # Оценка на каждой эпохе
    per_device_train_batch_size=16,     # Размер батча
    per_device_eval_batch_size=16,      # Размер батча для оценки
    num_train_epochs=5,                 # Количество дополнительных эпох
    weight_decay=0.01,                  # Уменьшение весов для регуляризации
    logging_dir='./logs',               # Директория для логов
)

# Создаем новый Trainer для дообучения
new_trainer = Trainer(
    model=model,
    args=new_training_args,
    train_dataset=tokenized_dataset,    # Ваш тренировочный набор
    eval_dataset=tokenized_dataset,     # Ваш тестовый набор (при необходимости)
)

# Продолжаем обучение
new_trainer.train()

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.019748
2,No log,0.010517
3,No log,0.006660
4,No log,0.004211
5,No log,0.003384


TrainOutput(global_step=5, training_loss=0.01567746847867966, metrics={'train_runtime': 10.3547, 'train_samples_per_second': 0.483, 'train_steps_per_second': 0.483, 'total_flos': 1306495595520.0, 'train_loss': 0.01567746847867966, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./saved_model_17")
tokenizer.save_pretrained("./saved_model_17")

('./saved_model_17/tokenizer_config.json',
 './saved_model_17/special_tokens_map.json',
 './saved_model_17/vocab.txt',
 './saved_model_17/added_tokens.json',
 './saved_model_17/tokenizer.json')

In [ ]:
# Заново создаем ваш тренировочный датасет (как и ранее)
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Определяем новые параметры для дообучения
new_training_args = TrainingArguments(
    output_dir="./results",             # Директория для сохранения моделей
    evaluation_strategy="epoch",        # Оценка на каждой эпохе
    per_device_train_batch_size=16,     # Размер батча
    per_device_eval_batch_size=16,      # Размер батча для оценки
    num_train_epochs=7,                 # Количество дополнительных эпох
    weight_decay=0.01,                  # Уменьшение весов для регуляризации
    logging_dir='./logs',               # Директория для логов
)

# Создаем новый Trainer для дообучения
new_trainer = Trainer(
    model=model,
    args=new_training_args,
    train_dataset=tokenized_dataset,    # Ваш тренировочный набор
    eval_dataset=tokenized_dataset,     # Ваш тестовый набор (при необходимости)
)

# Продолжаем обучение
new_trainer.train()

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.012483
2,No log,0.002598
3,No log,0.003274
4,No log,0.005076
5,No log,0.001784
6,No log,0.001601
7,No log,0.001555


TrainOutput(global_step=7, training_loss=0.0054203250578471595, metrics={'train_runtime': 11.3602, 'train_samples_per_second': 0.616, 'train_steps_per_second': 0.616, 'total_flos': 1829093833728.0, 'train_loss': 0.0054203250578471595, 'epoch': 7.0})

In [ ]:
model.save_pretrained("./saved_model_24")
tokenizer.save_pretrained("./saved_model_24")

('./saved_model_24/tokenizer_config.json',
 './saved_model_24/special_tokens_map.json',
 './saved_model_24/vocab.txt',
 './saved_model_24/added_tokens.json',
 './saved_model_24/tokenizer.json')

In [ ]:
model_12 = BertForTokenClassification.from_pretrained("./saved_model_12")
tokenizer_12 = BertTokenizerFast.from_pretrained("./saved_model_12")

model_17 = BertForTokenClassification.from_pretrained("./saved_model_17")
tokenizer_17 = BertTokenizerFast.from_pretrained("./saved_model_17")

model_24 = BertForTokenClassification.from_pretrained("./saved_model_24")
tokenizer_24 = BertTokenizerFast.from_pretrained("./saved_model_24")

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_webpage(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text()  # Extract raw text
            text = text.split()
            text = [word.strip() for word in text]
            text = [word for word in text if word != '']
            return ' '.join(text)
        else:
            print(f"Failed to retrieve page: {url}, \nstatus code={response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {str(e)}")
        return None

In [ ]:
from transformers import pipeline

# Создаем NER pipeline с использованием дообученной модели
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=device)

# Пример текста
text = scrape_webpage("https://www.factorybuys.com.au/products/euro-top-mattress-king")
print(text[:100])

# Извлечение именованных сущностей
entities = ner_pipeline(text)


# Вывод финальных сущностей
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")


Factory Buys 32cm Euro Top Mattress - King Skip to content FREE SHIPPING ON MATTRESSES* BED FRAMES F
Entity: Factory, Label: LABEL_1
Entity: Buy, Label: LABEL_2
Entity: ##s, Label: LABEL_2
Entity: 32, Label: LABEL_2
Entity: ##c, Label: LABEL_2
Entity: ##m, Label: LABEL_2
Entity: Euro, Label: LABEL_2
Entity: Top, Label: LABEL_2
Entity: Matt, Label: LABEL_2
Entity: ##ress, Label: LABEL_2
Entity: -, Label: LABEL_2
Entity: King, Label: LABEL_2
Entity: Ski, Label: LABEL_0
Entity: ##p, Label: LABEL_0
Entity: to, Label: LABEL_0
Entity: content, Label: LABEL_0
Entity: F, Label: LABEL_0
Entity: ##RE, Label: LABEL_0
Entity: ##E, Label: LABEL_0
Entity: SH, Label: LABEL_0
Entity: ##IP, Label: LABEL_0
Entity: ##PI, Label: LABEL_0
Entity: ##NG, Label: LABEL_0
Entity: ON, Label: LABEL_0
Entity: MA, Label: LABEL_0
Entity: ##TT, Label: LABEL_0
Entity: ##RE, Label: LABEL_0
Entity: ##SS, Label: LABEL_0
Entity: ##ES, Label: LABEL_0
Entity: *, Label: LABEL_0
Entity: B, Label: LABEL_0
Entity: ##ED, Label: L

### Compare 3 models:

In [ ]:
from transformers import pipeline

# Создаем NER pipeline с использованием дообученной модели
ner_pipeline = pipeline("ner", model=model_12, tokenizer=tokenizer, device=device)

# Пример текста
text = scrape_webpage("https://www.royaloakfurniture.co.uk/products/pop-bench")
if '£450.00' in text:
  print(True)

# # Извлечение именованных сущностей
# entities = ner_pipeline(text)


# # Вывод финальных сущностей
# for entity in entities:
#     if entity['word'][0].startswith('You'):
#       print(f"Entity: {entity['word']}, Label: {entity['entity']}")


NameError: name 'model_12' is not defined